In [5]:
import numpy as np
import pandas as pd
import io
import sys
from tqdm import tqdm

import torch
import torch.nn as nn

from sklearn.utils import class_weight

sys.path.insert(0, '../helpers/')
from normalize import normalize

ModuleNotFoundError: No module named 'pandas'

In [6]:
dataset = pd.read_csv('../data/train.txt', sep='\t')
dataset['turn1'] = dataset['turn1'].apply(normalise)
dataset['turn2'] = dataset['turn2'].apply(normalise)
dataset['turn3'] = dataset['turn3'].apply(normalise)


ValueError: Item wrong length 1 instead of 30160.

In [9]:
dataset[ dataset['id'] == 700 ]

,id,turn1,turn2,turn3,label
700,700,no problem i'll laugh,"here u go "" joke "" :p :p",haha,happy
